In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path

home = str(Path.home())
if home == '/home/hess': 
    %cd /home/hess/projects/weather-gan

if home == '/home/ftei-dsw': 
    %cd /home/ftei-dsw/projects/weather-gan

/home/ftei-dsw/projects/weather-gan


In [2]:
import torch
from pytorch_lightning.trainer import Trainer
from pytorch_lightning.loggers import TensorBoardLogger

from src.model import Generator, Discriminator
from src.trainer import WeatherGenerator
from src.data import DataModule
from src.utils import get_version, show_config
from src.configuration import Config
from src.trainer import training


In [3]:
'Checkpoint path: /home/ftei-dsw/data/weather-gan/checkpoints/weather-gan/16h_29m_18s/1bfcabc2-b67f-11ec-883b-845cf3c98452'

'Checkpoint path: /home/ftei-dsw/data/weather-gan/checkpoints/weather-gan/16h_29m_18s/1bfcabc2-b67f-11ec-883b-845cf3c98452'

In [17]:
import multiprocessing
multiprocessing.cpu_count()

12

# start training

In [18]:
training(Config(), enable_profiler=True)

ModelCheckpoint(save_last=True, save_top_k=-1, monitor=None) will duplicate the last checkpoint saved.


Running model: weather-gan/2022_04_27/17_37_46/fcc19134-c63f-11ec-ae2b-845cf3c98452
Checkpoint path: /home/ftei-dsw/data/weather-gan/checkpoints//weather-gan/2022_04_27/17_37_46/fcc19134-c63f-11ec-ae2b-845cf3c98452
################ configuration ################
-------------------- paths --------------------
tensorbard: /home/ftei-dsw/data/weather-gan/tensorboard/
checkpoint: /home/ftei-dsw/data/weather-gan/checkpoints/
config: /home/ftei-dsw/data/weather-gan/config-files/
results: /home/ftei-dsw/data/weather-gan/results/
input: /home/ftei-dsw/data/weather-gan/datasets/monthly_gfdl_historical.nc
target: /home/ftei-dsw/data/weather-gan/datasets/daily_gfdl_historical.nc
-------------------- splits  -------------------
training:   1950 - 2000
validation: 2001 - 2001
test:       2001 - 2001
------------------ hyperparams -----------------
num workers: 8
prefetch factor: 2
epochs: 3
train batch size: 32
test batch size: 64
lr: 0.0002
latent dim: 8
generator channels: 64
generator num. resi

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type          | Params
------------------------------------------------
0 | generator     | Generator     | 6.7 M 
1 | discriminator | Discriminator | 693 K 
------------------------------------------------
7.4 M     Trainable params
0         Non-trainable params
7.4 M     Total params
29.480    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

FIT Profiler Report

Action                             	|  Mean duration (s)	|Num calls      	|  Total time (s) 	|  Percentage %   	|
--------------------------------------------------------------------------------------------------------------------------------------
Total                              	|  -              	|_              	|  165.67         	|  100 %          	|
--------------------------------------------------------------------------------------------------------------------------------------
run_training_epoch                 	|  54.785         	|3              	|  164.36         	|  99.205         	|
run_training_batch                 	|  0.08652        	|1743           	|  150.8          	|  91.026         	|
training_step_and_backward         	|  0.052974       	|1743           	|  92.334         	|  55.733         	|
optimizer_step_with_closure_1      	|  0.061203       	|1452           	|  88.866         	|  53.64          	|
model_forward                      

# Debug